In [1]:
import pandas as pd
import numpy as np
import requests
import string
from bs4 import BeautifulSoup


In [290]:
pd.set_option('max_columns', 999)

In [6]:
#import list of unique fight urls
fight_urls = list(pd.read_csv('fight_urls.csv')['fight_url'])

In [539]:
def test():
    url = fight_urls[0]
    fight_url = 'http://www.fightmetric.com/fight-details/{}'.format(url)
    response = requests.get(fight_url)
    soup = BeautifulSoup(response.content, 'lxml')
    sig_stats=soup.find_all('tbody', {'class':'b-fight-details__table-body'})
    return sig_stats

In [544]:
fight_urls[0]

'b33e6a1d4911cf87'

In [525]:
def resultStats(result):
    fight_dict = {}
    fighters = result.find_all('div', {'class': 'b-fight-details__person'})
    try:
        fight_dict['fighter_a'] = fighters[0].find('a')['href'].split('/')[-1]
    except:
        name = fighters[0].find('span').text.strip()
        fight_dict['fighter_a'] = name.split(' ')[0].lower() + '_' + name.split(' ')[1].lower()
    try:
        fight_dict['fighter_b'] = fighters[1].find('a')['href'].split('/')[-1]
    except:
        name = fighters[1].find('span').text.strip()
        fight_dict['fighter_b'] = name.split(' ')[0].lower() + '_' + name.split(' ')[1].lower()
    fight_dict['fighter_a_result'] = fighters[0].find('i').text.strip()
    fight_dict['fighter_b_result'] = fighters[1].find('i').text.strip()
    return fight_dict

In [586]:
def summaryStats(summary, fight_dict):
    fight_dict['weight_class'] = summary.find('i').text.strip()
    elements = summary.find_all('i')[3:]
    fight_dict['method'] = elements[0].text.strip()
    fight_dict['round'] = elements[1].text.strip()[-1]
    fight_dict['time'] = elements[3].text.strip().split(' ')[-1]
    fight_dict['num_rounds'] = elements[5].text.split('Rnd')[0][-2]
    fight_dict['referee'] = elements[7].text.split('\n')[-3].strip()
    try:
        fight_dict['score1'] = elements[11].text.split('\n')[-2].strip()
        fight_dict['score2'] = elements[12].text.split('\n')[-2].strip()
        fight_dict['score3'] = elements[13].text.split('\n')[-2].strip()
    except:
        pass
    return fight_dict

In [387]:
def entireFightStats(soup, fight_dict):
    overall_stats = soup.find_all('section', {'class':'b-fight-details__section js-fight-section'})[1].find_all('tr')[1].find_all('td')
    top = 'fighter_a'
    bottom = 'fighter_b'
    top_ind = 2
    bottom_ind = 5
    fight_dict['total' + '_' + top + '_'+ 'kd'] = overall_stats[1].text.split('\n')[top_ind].strip()
    fight_dict['total' + '_' + bottom + '_'+ 'kd'] = overall_stats[1].text.split('\n')[bottom_ind].strip()
    fight_dict['total' + '_' + top + '_'+ 'sub_att'] = overall_stats[7].text.split('\n')[top_ind].strip()
    fight_dict['total' + '_' + bottom + '_'+ 'sub_att'] = overall_stats[7].text.split('\n')[bottom_ind].strip()
    fight_dict['total' + '_' + top + '_'+ 'passes'] = overall_stats[8].text.split('\n')[top_ind].strip()
    fight_dict['total' + '_' + bottom + '_'+ 'passes'] = overall_stats[8].text.split('\n')[bottom_ind].strip()
    fight_dict['total' + '_' + top + '_'+ 'reverses'] = overall_stats[9].text.split('\n')[top_ind].strip()
    fight_dict['total' + '_' + bottom + '_'+ 'reverses'] = overall_stats[9].text.split('\n')[bottom_ind].strip()
    fight_dict['total' + '_' + top + '_'+ 'ssl'] = overall_stats[2].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'ssa'] = overall_stats[2].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'ssl']  = overall_stats[2].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'ssa']  = overall_stats[2].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + top + '_'+ 'tsl'] = overall_stats[4].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'tsa'] = overall_stats[4].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'tsl']  = overall_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'tsa']  = overall_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + top + '_'+ 'tdl'] = overall_stats[5].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'tda'] = overall_stats[5].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'tdl']  = overall_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'tda']  = overall_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    return fight_dict

In [549]:
def entireSigStats(soup, fight_dict):
    sig_stats = soup.find_all('tbody', {'class':'b-fight-details__table-body'})[2].find('tr').find_all('td')
    top = 'fighter_a'
    bottom = 'fighter_b'
    top_ind = 2
    bottom_ind = 5
    fight_dict['total' + '_' + top + '_'+ 'hsl'] = sig_stats[3].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'hsa'] = sig_stats[3].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'hsl'] = sig_stats[3].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'hsa'] = sig_stats[3].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + top + '_'+ 'bsl'] = sig_stats[4].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'bsa'] = sig_stats[4].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'bsl'] = sig_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'bsa'] = sig_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + top + '_'+ 'lsl'] = sig_stats[5].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'lsa'] = sig_stats[5].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'lsl'] = sig_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'lsa'] = sig_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + top + '_'+ 'dsl'] = sig_stats[6].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'dsa'] = sig_stats[6].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'dsl'] = sig_stats[6].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'dsa'] = sig_stats[6].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + top + '_'+ 'csl'] = sig_stats[7].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'csa'] = sig_stats[7].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'csl'] = sig_stats[7].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'csa'] = sig_stats[7].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + top + '_'+ 'gsl'] = sig_stats[8].text.split('\n')[top_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + top + '_'+ 'gsa'] = sig_stats[8].text.split('\n')[top_ind].strip().split(' ')[-1]
    fight_dict['total' + '_' + bottom + '_'+ 'gsl'] = sig_stats[8].text.split('\n')[bottom_ind].strip().split(' ')[0]
    fight_dict['total' + '_' + bottom + '_'+ 'gsa'] = sig_stats[8].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    return fight_dict

In [537]:
def totalStats(totals, fight_dict):
    top = 'fighter_a'
    bottom = 'fighter_b'
    rounds = totals.find('tbody').find_all('tr')
    top_ind = 2
    bottom_ind = 5
    for i, Round in enumerate(rounds):
        r = str(i + 1)
        round_stats = Round.find_all('td')
        fight_dict[r+ '_'+ top + '_'+ 'kd'] = round_stats[1].text.split('\n')[top_ind].strip()
        fight_dict[r+ '_'+ bottom + '_'+ 'kd'] = round_stats[1].text.split('\n')[bottom_ind].strip()
        fight_dict[r+ '_'+ top + '_'+ 'ssl'] = round_stats[2].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r+ '_'+ top + '_'+ 'ssa'] = round_stats[2].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r+ '_'+ bottom + '_'+ 'ssl'] = round_stats[2].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r+ '_'+ bottom + '_'+ 'ssa'] = round_stats[2].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r+ '_'+ top + '_'+ 'tsl'] = round_stats[4].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r+ '_'+ top + '_'+ 'tsa'] = round_stats[4].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r+ '_'+ bottom + '_'+ 'tsl'] = round_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r+ '_'+ bottom + '_'+ 'tsa'] = round_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r+ '_'+ top + '_'+ 'tdl'] = round_stats[5].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r+ '_'+ top + '_'+ 'tda'] = round_stats[5].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r+ '_'+ bottom + '_'+ 'tdl'] = round_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r+ '_'+ bottom + '_'+ 'tda'] = round_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r+ '_'+ top + '_'+ 'sub_att'] = round_stats[7].text.split('\n')[top_ind].strip()
        fight_dict[r+ '_'+ bottom + '_'+ 'sub_att'] = round_stats[7].text.split('\n')[bottom_ind].strip()
        fight_dict[r+ '_'+ top + '_'+ 'passes'] = round_stats[8].text.split('\n')[top_ind].strip()
        fight_dict[r+ '_'+ bottom + '_'+ 'passes'] = round_stats[8].text.split('\n')[bottom_ind].strip()
        fight_dict[r+ '_'+ top + '_'+ 'reverses'] = round_stats[9].text.split('\n')[top_ind].strip()
        fight_dict[r+ '_'+ bottom + '_'+ 'reverses'] = round_stats[9].text.split('\n')[bottom_ind].strip()
    return fight_dict

In [536]:
def significantStrikes(sig_strikes, fight_dict):
    top = 'fighter_a'
    bottom = 'fighter_b'
    rounds = sig_strikes.find('tbody').find_all('tr')
    top_ind = 2
    bottom_ind = 5
    for i, Round in enumerate(rounds):
        r = str(i+1)
        round_stats = Round.find_all('td')
        fight_dict[r + '_' + top + '_' + 'hsl'] = round_stats[3].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r + '_' + top + '_' + 'hsa'] = round_stats[3].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + bottom + '_' + 'hsl'] = round_stats[3].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r + '_' + bottom + '_' + 'hsa'] = round_stats[3].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + top + '_' + 'bsl'] = round_stats[4].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r + '_' + top + '_' + 'bsa'] = round_stats[4].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + bottom + '_' + 'bsl'] = round_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r + '_' + bottom + '_' + 'bsa'] = round_stats[4].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + top + '_' + 'lsl'] = round_stats[5].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r + '_' + top + '_' + 'lsa'] = round_stats[5].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + bottom + '_' + 'lsl'] = round_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r + '_' + bottom + '_' + 'lsa'] = round_stats[5].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + top + '_' + 'dsl'] = round_stats[6].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r + '_' + top + '_' + 'dsa'] = round_stats[6].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + bottom + '_' + 'dsl'] = round_stats[6].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r + '_' + bottom + '_' + 'dsa'] = round_stats[6].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + top + '_' + 'csl'] = round_stats[7].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r + '_' + top + '_' + 'csa'] = round_stats[7].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + bottom + '_' + 'csl'] = round_stats[7].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r + '_' + bottom + '_' + 'csa'] = round_stats[7].text.split('\n')[bottom_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + top + '_' + 'gsl'] = round_stats[8].text.split('\n')[top_ind].strip().split(' ')[0]
        fight_dict[r + '_' + top + '_' + 'gsa'] = round_stats[8].text.split('\n')[top_ind].strip().split(' ')[-1]
        fight_dict[r + '_' + bottom + '_' + 'gsl'] = round_stats[8].text.split('\n')[bottom_ind].strip().split(' ')[0]
        fight_dict[r + '_' + bottom + '_' + 'gsa'] = round_stats[8].text.split('\n')[bottom_ind].strip().split(' ')[-1]
    return fight_dict

        

In [672]:
def Driver(url):
    #takes in a fight url and returns a dictionary of ALL the stats from the fight.
    #this includes round by round stats as well as overall totals
    fight_url = 'http://www.fightmetric.com/fight-details/{}'.format(url)
    response = requests.get(fight_url)
    soup = BeautifulSoup(response.content, 'lxml')
    result = soup.find('div', {'class': 'b-fight-details__persons clearfix'})
    fight_dict = resultStats(result)
    summary = soup.find('div', {'class':'b-fight-details__fight'})
    fight_dict = summaryStats(summary, fight_dict)
    fight_dict['date'] = dict_dates[url]
    try:
        fight_dict = entireFightStats(soup, fight_dict)
        fight_dict = entireSigStats(soup, fight_dict)
        totals, sig_strikes = soup.find_all('table', {'class': 'b-fight-details__table js-fight-table'})
        fight_dict = totalStats(totals, fight_dict)
        fight_dict = significantStrikes(sig_strikes, fight_dict)
    except:
        pass
    
    return fight_dict

#check to see if the script is written correctly
df = pd.DataFrame([d])

cols_a = []
cols_b = []
for key in d.keys():
    if 'fighter_a' in key:
        cols_a.append(key)
    elif 'fighter_b' in key:
        cols_b.append(key)

unifiedA = {x: x.replace('_a_', '_') for x in cols_a }

unifiedB = {x: x.replace('_b_', '_') for x in cols_b }

A = df[[x for x in df.columns if 'fighter_a' in x]]

B = df[[x for x in df.columns if 'fighter_b' in x]]

A.rename_axis(unifiedA, axis=1, inplace=True)

B.rename_axis(unifiedB, axis=1, inplace=True)

data = pd.concat([A,B])

In [415]:
df = pd.read_csv('complete_fighters.csv')

In [420]:
fighter_urls = df.url.unique()

In [421]:
dict_dates = {}


#add date for each fight
for i,url in enumerate(fighter_urls):
    if i%25==0:
        print i
    fighter_url = 'http://www.fightmetric.com/fighter-details/{}'.format(url)
    response = requests.get(fighter_url)
    soup = BeautifulSoup(response.content, 'lxml')
    fighters_fights = soup.find('tbody', {'class': 'b-fight-details__table-body'}).find_all('tr')[1:]
    for fight in fighters_fights:
        try:
            fight_url = str(fight).split("doNav('")[1].split("'")[0].split('/')[-1]
            date = fight.find_all('td')[6].text.split('\n')[-3].strip()
            if fight_url in dict_dates:
                pass
            else:
                dict_dates[fight_url] = date
        except:
            pass


In [444]:
url_df = pd.read_csv('fight_urls.csv')

In [594]:
url_df[['fight_url']].to_csv('only_urls.csv', index=False)

In [450]:
url_matrix = url_df.as_matrix()

In [561]:
fight_rows = []

In [573]:
#building matrix without multi processing
def timeA():
    for i,row in enumerate(url_matrix[0:10]):
        Driver(row[0])
    

In [576]:
#building matrix without multi processing
def timeB():
    for i,row in enumerate(url_matrix[50:60]):
        Driver(row[0])
    

In [562]:
#Below is where I experiment with multi-threading

In [634]:
import csv
import threading
import timeit
from multiprocessing import Pool as P # This is a thread-based Pool
from multiprocessing import cpu_count

In [575]:
timeit.timeit(timeA,number = 1)

40.43817400932312

In [577]:
timeit.timeit(timeB,number = 1)

44.86422300338745

In [579]:
# on average each fight takes 4.2 seconds to process


In [591]:
Driver('9be14b958737f074')

{'date': u'Dec. 04, 2010',
 'fighter_a': u'tom_aaron',
 'fighter_a_result': u'L',
 'fighter_b': u'matt_ricehouse',
 'fighter_b_result': u'W',
 'method': u'Decision - Unanimous',
 'num_rounds': u'3',
 'referee': u'',
 'round': u'3',
 'score1': u'28 - 29.',
 'score2': u'28 - 29.',
 'score3': u'27 - 30.',
 'time': u'5:00',
 'weight_class': u'Lightweight Bout'}

In [712]:
url_list = [list(x)[0] for x in url_df[['fight_url']].iloc[:88].values]

In [ ]:
pool = P(cpu_count() * 2)

thiss = pool.map(Driver, url_list)

In [682]:
pd.DataFrame(thiss).head()

,1_fighter_a_bsa,1_fighter_a_bsl,1_fighter_a_csa,1_fighter_a_csl,1_fighter_a_dsa,1_fighter_a_dsl,1_fighter_a_gsa,1_fighter_a_gsl,1_fighter_a_hsa,1_fighter_a_hsl,1_fighter_a_kd,1_fighter_a_lsa,1_fighter_a_lsl,1_fighter_a_passes,1_fighter_a_reverses,1_fighter_a_ssa,1_fighter_a_ssl,1_fighter_a_sub_att,1_fighter_a_tda,1_fighter_a_tdl,1_fighter_a_tsa,1_fighter_a_tsl,1_fighter_b_bsa,1_fighter_b_bsl,1_fighter_b_csa,1_fighter_b_csl,1_fighter_b_dsa,1_fighter_b_dsl,1_fighter_b_gsa,1_fighter_b_gsl,1_fighter_b_hsa,1_fighter_b_hsl,1_fighter_b_kd,1_fighter_b_lsa,1_fighter_b_lsl,1_fighter_b_passes,1_fighter_b_reverses,1_fighter_b_ssa,1_fighter_b_ssl,1_fighter_b_sub_att,1_fighter_b_tda,1_fighter_b_tdl,1_fighter_b_tsa,1_fighter_b_tsl,2_fighter_a_bsa,2_fighter_a_bsl,2_fighter_a_csa,2_fighter_a_csl,2_fighter_a_dsa,2_fighter_a_dsl,2_fighter_a_gsa,2_fighter_a_gsl,2_fighter_a_hsa,2_fighter_a_hsl,2_fighter_a_kd,2_fighter_a_lsa,2_fighter_a_lsl,2_fighter_a_passes,2_fighter_a_reverses,2_fighter_a_ssa,2_fighter_a_ssl,2_fighter_a_sub_att,2_fighter_a_tda,2_fighter_a_tdl,2_fighter_a_tsa,2_fighter_a_tsl,2_fighter_b_bsa,2_fighter_b_bsl,2_fighter_b_csa,2_fighter_b_csl,2_fighter_b_dsa,2_fighter_b_dsl,2_fighter_b_gsa,2_fighter_b_gsl,2_fighter_b_hsa,2_fighter_b_hsl,2_fighter_b_kd,2_fighter_b_lsa,2_fighter_b_lsl,2_fighter_b_passes,2_fighter_b_reverses,2_fighter_b_ssa,2_fighter_b_ssl,2_fighter_b_sub_att,2_fighter_b_tda,2_fighter_b_tdl,2_fighter_b_tsa,2_fighter_b_tsl,3_fighter_a_bsa,3_fighter_a_bsl,3_fighter_a_csa,3_fighter_a_csl,3_fighter_a_dsa,3_fighter_a_dsl,3_fighter_a_gsa,3_fighter_a_gsl,3_fighter_a_hsa,3_fighter_a_hsl,3_fighter_a_kd,3_fighter_a_lsa,3_fighter_a_lsl,3_fighter_a_passes,3_fighter_a_reverses,3_fighter_a_ssa,3_fighter_a_ssl,3_fighter_a_sub_att,3_fighter_a_tda,3_fighter_a_tdl,3_fighter_a_tsa,3_fighter_a_tsl,3_fighter_b_bsa,3_fighter_b_bsl,3_fighter_b_csa,3_fighter_b_csl,3_fighter_b_dsa,3_fighter_b_dsl,3_fighter_b_gsa,3_fighter_b_gsl,3_fighter_b_hsa,3_fighter_b_hsl,3_fighter_b_kd,3_fighter_b_lsa,3_fighter_b_lsl,3_fighter_b_passes,3_fighter_b_reverses,3_fighter_b_ssa,3_fighter_b_ssl,3_fighter_b_sub_att,3_fighter_b_tda,3_fighter_b_tdl,3_fighter_b_tsa,3_fighter_b_tsl,4_fighter_a_bsa,4_fighter_a_bsl,4_fighter_a_csa,4_fighter_a_csl,4_fighter_a_dsa,4_fighter_a_dsl,4_fighter_a_gsa,4_fighter_a_gsl,4_fighter_a_hsa,4_fighter_a_hsl,4_fighter_a_kd,4_fighter_a_lsa,4_fighter_a_lsl,4_fighter_a_passes,4_fighter_a_reverses,4_fighter_a_ssa,4_fighter_a_ssl,4_fighter_a_sub_att,4_fighter_a_tda,4_fighter_a_tdl,4_fighter_a_tsa,4_fighter_a_tsl,4_fighter_b_bsa,4_fighter_b_bsl,4_fighter_b_csa,4_fighter_b_csl,4_fighter_b_dsa,4_fighter_b_dsl,4_fighter_b_gsa,4_fighter_b_gsl,4_fighter_b_hsa,4_fighter_b_hsl,4_fighter_b_kd,4_fighter_b_lsa,4_fighter_b_lsl,4_fighter_b_passes,4_fighter_b_reverses,4_fighter_b_ssa,4_fighter_b_ssl,4_fighter_b_sub_att,4_fighter_b_tda,4_fighter_b_tdl,4_fighter_b_tsa,4_fighter_b_tsl,5_fighter_a_bsa,5_fighter_a_bsl,5_fighter_a_csa,5_fighter_a_csl,5_fighter_a_dsa,5_fighter_a_dsl,5_fighter_a_gsa,5_fighter_a_gsl,5_fighter_a_hsa,5_fighter_a_hsl,5_fighter_a_kd,5_fighter_a_lsa,5_fighter_a_lsl,5_fighter_a_passes,5_fighter_a_reverses,5_fighter_a_ssa,5_fighter_a_ssl,5_fighter_a_sub_att,5_fighter_a_tda,5_fighter_a_tdl,5_fighter_a_tsa,5_fighter_a_tsl,5_fighter_b_bsa,5_fighter_b_bsl,5_fighter_b_csa,5_fighter_b_csl,5_fighter_b_dsa,5_fighter_b_dsl,5_fighter_b_gsa,5_fighter_b_gsl,5_fighter_b_hsa,5_fighter_b_hsl,5_fighter_b_kd,5_fighter_b_lsa,5_fighter_b_lsl,5_fighter_b_passes,5_fighter_b_reverses,5_fighter_b_ssa,5_fighter_b_ssl,5_fighter_b_sub_att,5_fighter_b_tda,5_fighter_b_tdl,5_fighter_b_tsa,5_fighter_b_tsl,date,fighter_a,fighter_a_result,fighter_b,fighter_b_result,method,num_rounds,referee,round,score1,score2,score3,time,total_fighter_a_bsa,total_fighter_a_bsl,total_fighter_a_csa,total_fighter_a_csl,total_fighter_a_dsa,total_fighter_a_dsl,total_fighter_a_gsa,total_fighter_a_gsl,total_fighter_a_hsa,total_fighter_a_hsl,total_fighter_a_kd,total_fighter_a_lsa,total_fighter

In [662]:
with open('Test.csv', 'ab') as f:
    writeFile = csv.writer(f)
    for thing in newname:
        writeFile.writerow(thing)

Error: sequence expected

In [660]:
Driver('9afb09b16993180b')

9afb09b16993180b


{'1_fighter_a_bsa': u'6',
 '1_fighter_a_bsl': u'5',
 '1_fighter_a_csa': u'5',
 '1_fighter_a_csl': u'2',
 '1_fighter_a_dsa': u'23',
 '1_fighter_a_dsl': u'12',
 '1_fighter_a_gsa': u'2',
 '1_fighter_a_gsl': u'2',
 '1_fighter_a_hsa': u'22',
 '1_fighter_a_hsl': u'9',
 '1_fighter_a_kd': u'0',
 '1_fighter_a_lsa': u'2',
 '1_fighter_a_lsl': u'2',
 '1_fighter_a_passes': u'1',
 '1_fighter_a_reverses': u'0',
 '1_fighter_a_ssa': u'30',
 '1_fighter_a_ssl': u'16',
 '1_fighter_a_sub_att': u'0',
 '1_fighter_a_tda': u'4',
 '1_fighter_a_tdl': u'2',
 '1_fighter_a_tsa': u'58',
 '1_fighter_a_tsl': u'43',
 '1_fighter_b_bsa': u'2',
 '1_fighter_b_bsl': u'1',
 '1_fighter_b_csa': u'3',
 '1_fighter_b_csl': u'1',
 '1_fighter_b_dsa': u'14',
 '1_fighter_b_dsl': u'2',
 '1_fighter_b_gsa': u'0',
 '1_fighter_b_gsl': u'0',
 '1_fighter_b_hsa': u'14',
 '1_fighter_b_hsl': u'1',
 '1_fighter_b_kd': u'0',
 '1_fighter_b_lsa': u'1',
 '1_fighter_b_lsl': u'1',
 '1_fighter_b_passes': u'0',
 '1_fighter_b_reverses': u'0',
 '1_fighter